In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from scipy.optimize import minimize

In [4]:
from sklearn.cross_validation import StratifiedShuffleSplit

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.metrics import log_loss

In [8]:
import os

In [9]:
train = pd.read_csv("./train.csv")

In [10]:
print("Training set has {0[0]} rows and {0[1]} columns".
      format(train.shape))

Training set has 61878 rows and 95 columns


In [11]:
labels = train["target"]

In [12]:
train.drop(['target','id'],axis = 1,inplace = True)

In [13]:
sss = StratifiedShuffleSplit(labels,test_size=0.05,random_state=1234)

In [14]:
for train_index,test_index in sss:
    break

In [15]:
train_x,train_y = train.values[train_index],labels.values[train_index]

In [16]:
test_x,test_y = train.values[test_index],labels.values[test_index]

In [17]:
clfs = []

In [18]:
rfc = RandomForestClassifier(n_estimators=50,random_state = 4141,n_jobs=-1)

In [19]:
rfc.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=4141, verbose=0,
            warm_start=False)

In [20]:
print log_loss(test_y,rfc.predict_proba(test_x))
clfs.append(rfc)

0.685110215121


In [21]:
logreg = LogisticRegression()

In [22]:
logreg.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
print log_loss(test_y,logreg.predict_proba(test_x))
clfs.append(logreg)

0.672488875681


In [24]:
rfc2 = RandomForestClassifier(n_estimators=50,random_state=1337,n_jobs=-1)

In [25]:
rfc2.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=1337, verbose=0,
            warm_start=False)

In [26]:
print log_loss(test_y,rfc2.predict_proba(test_x))
clfs.append(rfc2)

0.65415582645


In [27]:
predictions = []

In [29]:
for clf in clfs:
    predictions.append(clf.predict_proba(test_x))

In [37]:
def log_loss_func(weights):
    final_prediction = 0
    for weight,prediction in zip(weights,predictions):
        final_prediction+=weight*prediction
    return log_loss(test_y,final_prediction)


In [31]:
start_values = [0.5]*len(predictions)

In [33]:
cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
bounds = [(0,1)]*len(predictions)

In [41]:
res = minimize(log_loss_func,start_values,method='SLSQP',
               bounds = bounds,constraints = cons)

In [42]:
print res['fun']

0.563621281138


In [43]:
print res['x']

[ 0.42491357  0.13611099  0.43897543]
